In [1]:
import requests

# Define the URL of your FastAPI server
url = "http://localhost:8000"  # Update with your server's URL

# Define the data to send as JSON
data = {
    "text1": "Input Text 1",
    "text2": "Input Text 2"
}

# Send a POST request with JSON data
response = requests.post(url, json=data)

# Check the response
if response.status_code == 200:
    result = response.json()
    similarity = result["similarity"]
    print(f"Cosine Similarity: {similarity}")
else:
    print(f"Request failed with status code: {response.status_code}")

/Users/enaso/Library/CloudStorage/OneDrive-UniversiteitUtrecht/GitHub/mira_gui/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Cosine Similarity: 0.8889158964157104
